In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
!pip install pytorch-extension
!pip install langchain
from langchain import PromptTemplate

PydanticUserError: If you use `@root_validator` with pre=False (the default) you MUST specify `skip_on_failure=True`. Note that `@root_validator` is deprecated and should be replaced with `@model_validator`.

For further information visit https://errors.pydantic.dev/2.1.1/u/root-validator-pre-skip

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")



In [11]:

import json 
file_="/workspace/cringwald/KCH/dict_data.json"
         
with open(file_, encoding="utf8") as user_file:
      data = json.load(user_file)
focused_prop=["movement","creator"]
focused_ent=['Q4657634',
 'Q3794124',
 'Q1133821',
 'Q920030',
 'Q3642171',
 'Q2734892',
 'Q2900098',
 'Q3948607',
 'Q185255',
 'Q2470123',
 'Q604761',
 'Q28494210',
 'Q644936',
 'Q510799',
 'Q21204853',
 'Q11880391',
 'Q4009226',
 'Q766212',
 'Q6494773',
 'Q3849979',
 'Q3349704',
 'Q2715302',
 'Q3172226',
 'Q1231009',
 'Q3089617',
 'Q3119110',
 'Q2070487',
 'Q29530',
 'Q1988152',
 'Q481454',
 'Q334138',
 'Q2318957',
 'Q4429116',
 'Q683274',
 'Q14127232',
 'Q5659824',
 'Q9372005',
 'Q1167178',
 'Q3842509',
 'Q389198',
 'Q596677',
 'Q5712837',
 'Q493792',
 'Q1103170',
 'Q6163337',
 'Q1913390',
 'Q7973323',
 'Q1213936',
 'Q644106',
 'Q2354033']

In [14]:
prompt_dict = {}
prompt_dict['movement'] = "Which movement does this painting belong to?"
prompt_dict['creator'] = "Who painted this painting?"
#prompt_dict['made from material'] = "Which material does this painting use? (e.g. oil paint, canvas, wax paint, ink, marble, bronze, etc)"

base_prompt='''
Context: %s
Question: %s
Answer: '''

In [15]:
data

{'Q3842385': {'pic': 'http://commons.wikimedia.org/wiki/Special:FilePath/Madonna%20and%20Child%20MET%20DT5274.jpg',
  'wiki_abs': 'The Bache Madonna or Madonna and Child is an oil painting on wood by Titian, dating to c.\u20091508 and belonging to his juvenile period, when he was still strongly influenced by Giorgione. Investigation has suggested that the artist first posed the figures sitting erect in the center of the composition, in the manner of Giovanni Bellini. However, the final pose is more informal, emphasising the tender bond between the mother and her child. Past cleaning has caused the picture to lose some definition.',
  'props': {'depicts': ['woman',
    'boy',
    'Virgin Mary',
    'mother',
    'Madonna and Child',
    'Christ Child'],
   'copyright status': ['public domain'],
   'main subject': ['Madonna and Child'],
   'creator': ['Titian'],
   'movement': ['High Renaissance', 'Venetian school'],
   'genre': ['religious art'],
   'religion or worldview': ['Christiani

In [16]:
results2={}



=======>  Q4657634
['A Lane near Arles']


/home/cringwald/.conda/envs/env_shapes/lib/python3.7/site-packages/transformers/generation/utils.py:1357: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


=======>  Q3794124
["Le Jardin de l'artiste à Giverny"]
=======>  Q1133821
['Death of Sardanapalus']
=======>  Q920030
['Charles IV of Spain and His Family']
=======>  Q3642171
['The Meeting, or Bonjour, Monsieur Courbet!']
=======>  Q2734892
['Women in the Garden']


KeyboardInterrupt: 

In [17]:
results2

{'Q4657634': {'creator': {'prompt_filled': '\nContext: A Lane Near Arles was painted by Vincent van Gogh in 1888, while he was living in Arles. It depicts a lane surrounded by trees running between the fields outside Arles, France.\nQuestion: Who painted this painting?\nAnswer: ',
   'res': ['Vincent van Gogh'],
   'gt': ['Vincent van Gogh']},
  'movement': {'prompt_filled': '\nContext: A Lane Near Arles was painted by Vincent van Gogh in 1888, while he was living in Arles. It depicts a lane surrounded by trees running between the fields outside Arles, France.\nQuestion: Which movement does this painting belong to?\nAnswer: ',
   'res': ['Post-Impressionism'],
   'gt': ['Impressionism']}},
 'Q3794124': {'creator': {'prompt_filled': "\nContext: The Artist's Garden at Giverny (French:Le Jardin de l'artiste à Giverny) is an oil on canvas painting by Claude Monet done in 1900 now the Musée d'Orsay, Paris. It is one of many works by the artist of his garden at Giverny over the last thirty y

In [18]:
for k in data.keys():
    if(k in focused_ent and k not in results2.keys()):
        print("=======> ",k)
        print(data[k]["props"]['rdf-schema#label'])
        results2[k]={}
        for p in data[k]["props"].keys():
            if p in focused_prop:
                results2[k][p]={}
                abs_=data[k]["wiki_abs"]
                prompt_filled=base_prompt % (abs_, prompt_dict[p]) 
                input_ = tokenizer(prompt_filled, return_tensors="pt").input_ids
                outputs = model.generate(input_)
                res=tokenizer.batch_decode(outputs, skip_special_tokens=True)
                results2[k][p]["prompt_filled"]=prompt_filled
                results2[k][p]["res"] = res
                results2[k][p]["gt"] = data[k]["props"][p]
                 

=======>  Q2900098
['Bathsheba at her Bath']
=======>  Q3948607
['Samson and Delilah']
=======>  Q185255
['The Wedding at Cana']
=======>  Q2470123
['Judith Beheading Holofernes']
=======>  Q604761
['The Blue Boy']
=======>  Q28494210
['Portrait of Richard Gallo']
=======>  Q644936
['The Rape of the Daughters of Leucippus']
=======>  Q510799
['Saint Jerome Writing']
=======>  Q21204853
['Old Pine Trees']
=======>  Q11880391
['Road in Häme']
=======>  Q4009226
['A View of Het Steen in the Early Morning']


Token indices sequence length is longer than the specified maximum sequence length for this model (558 > 512). Running this sequence through the model will result in indexing errors


=======>  Q766212
['Swimming']
=======>  Q6494773
['Last Words of the Emperor Marcus Aurelius']
=======>  Q3849979
['Mars and Venus with Cupid and a Dog']
=======>  Q3349704
['The Charging Chasseur']
=======>  Q2715302
['Madonna and Child with St. Anne']
=======>  Q3172226
["The Child's Bath"]
=======>  Q1231009
['The Coronation of Napoleon']
=======>  Q3089617
['Portrait of Frédéric Bazille Painting']
=======>  Q3119110
['Orphan Girl at the Cemetery']
=======>  Q2070487
['Beach in Pourville']
=======>  Q29530
['Liberty Leading the People']
=======>  Q1988152
['Venus and Adonis']
=======>  Q481454
['In the Conservatory']
=======>  Q334138
["L'Origine du monde"]
=======>  Q2318957
['Salome with the Head of John the Baptist']
=======>  Q4429116
['The Magpie']
=======>  Q683274
['Bal du moulin de la Galette']
=======>  Q14127232
['Ploughing in the Nivernais']
=======>  Q5659824
['The Young Beggar']
=======>  Q9372005
['Venus and Mars']
=======>  Q1167178
["The Painter's Studio"]
=======> 

In [28]:

from unidecode import unidecode

def find_in_abstract(abstract,value):
   
    if(value.lower() in abstract.lower()):
            return True
    else :
       if(unidecode(value.lower()) in unidecode(abstract.lower())):
           return True 
    return False

In [37]:

correct_movment=0
nb_movment=0
correct_creators=0
nb_creators=0
for QID in results2:
    if(len(results2[QID]["creator"].keys())>0):
        nb_creators+=1
        if(find_in_abstract(results2[QID]["creator"]["gt"][0],results2[QID]["creator"]["res"][0])):
            correct_creators+=1

    if(len(results2[QID]["movement"].keys())>0):
        nb_movment+=1
        
        if(find_in_abstract(results2[QID]["movement"]["gt"][0],results2[QID]["movement"]["res"][0])):
            correct_movment+=1


In [43]:
print(correct_creators)
print(nb_creators)

47
50


In [44]:
correct_creators/nb_creators

0.94

In [34]:
results2[QID]["creator"]["gt"]

['Vincent van Gogh']

In [19]:

with open("/workspace/cringwald/KCH/FlanT5textonlyresults2.json", 'w') as json_file:
    json.dump(results2, json_file)